# Анализ базы данных сервиса для чтения книг по подписке

## Описание проекта

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Задачи анализа

Проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Ход анализа

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Загрузка и изучение данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Загрузим функцию, которая будет принимать на вход датафрейм, а на выходе она будет выводить всего нужные характеристики.

In [2]:
def load_info(df):
    display(df.head())
    print(df.info())
    display(df.describe())

In [3]:
books = '''select * from books'''
books = pd.read_sql_query(books, con = engine)
load_info(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
None


,book_id,author_id,num_pages,publisher_id
count,1000.000000,1000.000000,1000.00000,1000.000000
mean,500.500000,320.417000,389.11100,171.270000
std,288.819436,181.620172,229.39014,99.082685
min,1.000000,1.000000,14.00000,1.000000
25%,250.750000,162.750000,249.00000,83.000000
50%,500.500000,316.500000,352.00000,177.500000
75%,750.250000,481.000000,453.00000,258.000000
max,1000.000000,636.000000,2690.00000,340.000000


In [4]:
authors = '''select * from authors'''
authors = pd.read_sql_query(authors, con = engine)
load_info(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB
None


,author_id
count,636.000000
mean,318.500000
std,183.741666
min,1.000000
25%,159.750000
50%,318.500000
75%,477.250000
max,636.000000


In [5]:
publishers = '''select * from publishers'''
publishers = pd.read_sql_query(publishers, con = engine)
load_info(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB
None


,publisher_id
count,340.000000
mean,170.500000
std,98.293777
min,1.000000
25%,85.750000
50%,170.500000
75%,255.250000
max,340.000000


In [6]:
ratings = '''select * from ratings'''
ratings = pd.read_sql_query(ratings, con = engine)
load_info(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB
None


,rating_id,book_id,rating
count,6456.000000,6456.000000,6456.000000
mean,3228.500000,510.574195,3.928284
std,1863.831001,284.141636,0.943303
min,1.000000,1.000000,1.000000
25%,1614.750000,291.000000,3.000000
50%,3228.500000,506.000000,4.000000
75%,4842.250000,750.000000,5.000000
max,6456.000000,1000.000000,5.000000


In [7]:
reviews = '''select * from reviews'''
reviews = pd.read_sql_query(reviews, con = engine)
load_info(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB
None


,review_id,book_id
count,2793.000000,2793.000000
mean,1397.000000,504.693161
std,806.413976,288.472931
min,1.000000,1.000000
25%,699.000000,259.000000
50%,1397.000000,505.000000
75%,2095.000000,753.000000
max,2793.000000,1000.000000


## Анализ данных

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [8]:
t = '''
select COUNT(book_id)
from books
where publication_date >='2000-01-01'
'''
t = pd.read_sql_query(t, con = engine)
t

,count
0,821


После 1 января 2000 года вышло 821 книги.

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [14]:
t = '''
select title, b.book_id, count(distinct re.review_id), avg(ra.rating)
from books AS b
left join reviews AS re ON b.book_id=re.book_id
left join ratings AS ra ON re.book_id=ra.book_id
group by b.book_id, title
'''
t = pd.read_sql_query(t, con = engine)
t

,title,book_id,count,avg
0,'Salem's Lot,1,2,3.666667
1,1 000 Places to See Before You Die,2,1,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,3,4.666667
3,1491: New Revelations of the Americas Before C...,4,2,4.500000
4,1776,5,4,4.000000
...,...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),996,3,3.666667
996,Xenocide (Ender's Saga #3),997,3,3.400000
997,Year of Wonders,998,4,3.200000
998,You Suck (A Love Story #2),999,2,4.500000


### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [10]:
t = '''
select p.publisher, count(b.book_id)
from books AS b
join publishers AS p ON b.publisher_id=p.publisher_id
where b.num_pages >= 50
group by b.publisher_id, p.publisher
order by 2 desc
limit 1
'''
t = pd.read_sql_query(t, con = engine)
t

,publisher,count
0,Penguin Books,42


### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [11]:
t = '''
WITH t AS (select author, b.book_id, count(ra.rating), avg(ra.rating) as a
from books AS b
left join ratings AS ra ON b.book_id=ra.book_id
join authors AS a ON b.author_id=a.author_id
group by b.book_id, a.author
having count(ra.rating) >= 50
order by 1)
select t.author, avg(t.a)
from t
group by t.author
order by 2 desc
limit 1
'''
t = pd.read_sql_query(t, con = engine)
t

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [30]:
t = '''
with t as (select username, count(rating)
from ratings AS ra
group by username
having count(rating) > 50),
tt as (select t.username, count(re.review_id) as c
from t
join reviews AS re ON t.username=re.username
group by t.username)
select avg(tt.c)
from tt
'''
t = pd.read_sql_query(t, con = engine)
t

,avg
0,24.333333


## Вывод

В базе данных 5 таблиц с информацией о книгах, авторах, издателях, рейтингах и обзорах. Все таблицы связаны внутренними ключами.

В таблице книги 1000 значений. Для каждой книги посчитано количество обзоров и средняя оценка. 821 книга вышла после 1 января 2020 года.

В таблице авторов 636 значений. 

В таблице издательств 340 значений, из них Penguin Books выпустило наибольшее количество книг толще 50 страниц.

В таблице рейтинг 6456 значений. 

В таблице обзоры 2793 значений.

Определен автор с самой высокой средней оценкой книг (учитывая книги с 50 и более оценками) J.K. Rowling/Mary GrandPré 4.283844.

Посчитано среднее количество обзоров от пользователей, которые поставили больше 50 оценок = 24.333333.